<a href="https://colab.research.google.com/github/santiPP/-YOPROGRAMO/blob/main/DEVO-SCANNER9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import PyPDF2
import os
import textract
import openpyxl
import sqlite3

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def extract_text_from_txt(txt_path):
    with open(txt_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()
    return text

def extract_text_from_excel(excel_path):
    text = ""
    workbook = openpyxl.load_workbook(excel_path)
    for sheet in workbook:
        for row in sheet.iter_rows(values_only=True):
            for cell_value in row:
                if cell_value and isinstance(cell_value, str):
                    text += cell_value + "\n"
    return text

def extract_text_from_database(db_path, table_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    text = ""
    for row in rows:
        text += " ".join(str(cell) for cell in row) + "\n"
    conn.close()
    return text

def find_unique_information(text, pattern, part_number_pattern):
    information_matches = []
    pattern_matches = re.finditer(pattern, text)
    part_number_matches = re.finditer(part_number_pattern, text)

    for part_number_match, pattern_match in zip(part_number_matches, pattern_matches):
        part_number = part_number_match.group(1)
        pattern_info = pattern_match.group(1)
        information_matches.append((pattern_info, part_number))

    return information_matches

def main():
    source_path = r"C:\Users\Usuario\Documents\ENTREPRENEUR\ARTIFICIAL INTELLIGENCE\RENAULT\DOCUMENTOS\FFP940.txt"  # Replace with your file path
    file_extension = os.path.splitext(source_path)[1].lower()

    if file_extension == ".pdf":
        text = extract_text_from_pdf(source_path)
    elif file_extension == ".txt":
        text = extract_text_from_txt(source_path)
    elif file_extension == ".xlsx":
        text = extract_text_from_excel(source_path)
    elif file_extension == ".db":
        table_name = "PART NUMBERS & CHARACTERISTICS"  # Replace with the appropriate table name
        text = extract_text_from_database(source_path, table_name)
    else:
        print("Unsupported file format")
        return

    pattern = r'BE \[20230\]\s*•\s*([^\]]+)'
    part_number_pattern = r'(\d{6}) - ([A-Za-z]+)'

    information_matches = find_unique_information(text, pattern, part_number_pattern)

    if information_matches:
        print("Found Information:")
        for pattern_info, part_number in information_matches:
            print(f"Pattern: {pattern_info}, Part Number: {part_number}")
    else:
        print("No matching information found.")

if __name__ == "__main__":
    main()